Importing all required packages

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
IMAGE_SIZE = [224, 224,3]

Setting training and Testing path

In [5]:
train_path = '/content/drive/My Drive/transfer learning masks/covid masks/train'
valid_path = '/content/drive/My Drive/transfer learning masks/covid masks/test'

Preprocessing the Images

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 20,
                    zoom_range = 0.15,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.15,
                    horizontal_flip = True,
                    fill_mode="nearest")

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 919 images belonging to 2 classes.
Found 397 images belonging to 2 classes.


Using ResNet50 for Transfer Learning on our dataset

In [ ]:
Res = ResNet50(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
last_layer=Res.output            
last_layer = tf.keras.layers.AveragePooling2D(pool_size = (7,7))(last_layer)
last_layer = tf.keras.layers.Flatten()(last_layer)
last_layer = tf.keras.layers.Dense(256, activation = 'relu')(last_layer)
last_layer = tf.keras.layers.Dropout(0.5)(last_layer)
last_layer = tf.keras.layers.Dense(2, activation = 'softmax')(last_layer)

Accessing masked and unmasked images files from our folder

In [ ]:
from glob import glob
folders = glob('/content/drive/My Drive/transfer learning masks/covid masks/train/*')

Creating our model

In [ ]:
from keras.models import Model
model = Model(inputs=Res.input, outputs=last_layer)

Checking summary of our model for checking the layers in the network

In [ ]:
model.summary()


Compiling model with hyperparameters

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Fitting the model with our dataset

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=26,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set))

Evaluating the model

In [ ]:
 train_acc=model.evaluate(training_set,verbose=2)
 test_acc = model.evaluate(test_set, verbose=2)

In [ ]:
train_acc

In [ ]:
test_acc